In [17]:
import pandas as pd
import numpy as np
import os

path = 'data/covid_kaggle/covid19'
files = os.listdir(path)

sample_data = []
for file in files:
    data = pd.read_csv(path+'/'+file)
    sam_1 = data[data['retweet_count']>=10][data['lang']=='en'].sample(1000)
    sample_data.append(sam_1)
    sam_2 = data[data['retweet_count']<10][data['lang']=='en'].sample(1000)
    sample_data.append(sam_2)
sample_data = pd.concat(sample_data, axis=0)
sample_data.shape

C:\Users\ChuAI\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\ChuAI\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]


(90000, 22)

In [18]:
sample_data.iloc[:25000,:].to_csv('data/covid_kaggle/covid19/covid19_1.csv', index=False)
sample_data.iloc[25000:65000,:].to_csv('data/covid_kaggle/covid19/covid19_2.csv', index=False)
sample_data.iloc[65000:,:].to_csv('data/covid_kaggle/covid19/covid19_3.csv', index=False)

In [27]:
path_2 = 'data/covid_kaggle/covid19/covid19_emo'
files = os.listdir(path_2)

covid = []
for file in files:
    data = pd.read_csv(path_2+'/'+file)
    covid.append(data)
covid = pd.concat(covid, axis=0)
covid.shape

(90000, 24)

In [29]:
import ast
special_emos = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust']

covid['emo_words'] = covid['emo_words'].apply(lambda j: ast.literal_eval(j))
for emo in special_emos:
    covid[emo] = covid['emo_words'].apply(lambda ew: ew[emo])
covid['special_emos_sum'] = covid[special_emos].sum(axis=1)
for emo in special_emos:
    covid[emo] = covid[emo]/covid['special_emos_sum']

In [38]:
import re
covid['text_len'] = covid['text'].apply(lambda t: len(t))
covid['have_htag'] = covid['text'].apply(lambda t: 1 if re.search('#.*?', t, re.S|re.M) is not None else 0)
covid['have_url'] = covid['text'].apply(lambda t: 1 if re.search('http.*?', t, re.S|re.M) is not None else 0)
covid['have_at'] = covid['text'].apply(lambda t : (1 if re.search('@(.*)', t, re.S|re.M) is not None else 0))

In [39]:
import spacy
nlp = spacy.load('en_core_web_sm')

date_loc = []
def id_ent(text):
    doc = nlp(text)
    date = 0
    loc = 0
    labels = []
    for ent in doc.ents:
        labels.append(ent.label_)
    if 'DATE' in labels:
        date = 1
    if 'GPE' in labels:
        loc = 1
    date_loc.append([date, loc])
covid['text'].apply(id_ent)
covid[['have_date', 'have_loc']] = pd.DataFrame(date_loc)

In [41]:
covid.to_csv(path_2+'/covid_reg.csv', index=False)

In [42]:
elec_data = pd.read_csv(path_2+'/election_day_tweets_reg.csv')

In [50]:
sub_covid = covid[['user_id', 'retweet_count', 'followers_count', 'friends_count', 'status_id', 'emo_words', 'anger',
       'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise',
       'trust', 'special_emos_sum', 'text_len', 'have_htag', 'have_url',
       'have_at', 'have_date', 'have_loc']]
sub_covid.rename(columns={'status_id':'id'}, inplace=True)

C:\Users\ChuAI\Anaconda3\envs\pytorch\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [51]:
sub_elec_data = elec_data[['user.id', 'retweet_count', 'user.followers_count', 'user.friends_count', 'id', 'emo_words', 'anger',
       'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise',
       'trust', 'special_emos_sum', 'text_len', 'have_htag', 'have_url',
       'have_at', 'have_date', 'have_loc']]
sub_elec_data.rename(columns={'user.id':'user_id', 'user.followers_count':'followers_count', 'user.friends_count':'friends_count'}, inplace=True)

In [52]:
sub_elec_data['label'] = 'politic'
sub_covid['label'] = 'covid'
data = sub_elec_data.append(sub_covid, ignore_index=True)
data.head()

C:\Users\ChuAI\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\ChuAI\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,user_id,retweet_count,followers_count,friends_count,id,emo_words,anger,anticipation,disgust,fear,...,surprise,trust,special_emos_sum,text_len,have_htag,have_url,have_at,have_date,have_loc,label
0,22729901,8.0,86551.0,2674.0,795839414735732737,"{'anger': 1, 'anticipation': 2, 'disgust': 0, ...",0.142857,0.285714,0.0,0.0,...,0.142857,0.142857,7,139,1,0,1,1,1,politic
1,794779346,4.0,1689.0,344.0,795840310597193728,"{'anger': 0, 'anticipation': 1, 'disgust': 0, ...",0.000000,1.000000,0.0,0.0,...,0.000000,0.000000,1,142,1,1,1,0,0,politic
2,794779346,2.0,1689.0,344.0,795850304956661760,"{'anger': 0, 'anticipation': 0, 'disgust': 0, ...",NaN,NaN,NaN,NaN,...,NaN,NaN,0,138,1,1,1,0,0,politic
3,2311063720,2.0,3478.0,2951.0,795861571381710848,"{'anger': 1, 'anticipation': 1, 'disgust': 0, ...",0.142857,0.142857,0.0,0.0,...,0.142857,0.285714,7,139,1,1,0,1,0,politic
4,484040156,1.0,1224.0,2154.0,795866702974189568,"{'anger': 0, 'anticipation': 1, 'disgust': 0, ...",0.000000,0.500000,0.0,0.0,...,0.000000,0.500000,2,148,1,1,0,0,0,politic


In [53]:
data.to_csv(path_2+'/covid_elec_reg.csv', index=False)